# ==========================================
# UBER & LYFT PRICE ANALYSIS
# Dataset: Kaggle (693k cab rides + weather)
# Goal: Compare prices, analyze surge pricing, weather impact
# ==========================================
# https://www.kaggle.com/datasets/ravi72munde/uber-lyft-cab-prices
# ==========================================
# IMPORTS
# ==========================================

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print('Libraries are downloaded')



Libraries are downloaded


# ==========================================
# БЛОК 1: ЗАГРУЗКА ДАННЫХ
# ==========================================


In [7]:
# Загрузи cab_rides.csv
rides = pd.read_csv('data/cab_rides.csv')

# Загрузи weather.csv
weather = pd.read_csv('data/weather.csv')

print("📊 Датасеты загружены")
print(f"Rides: {rides.shape}")
print(f"Weather: {weather.shape}")

📊 Датасеты загружены
Rides: (693071, 10)
Weather: (6276, 8)


# ==========================================
# БЛОК 1.1: ИЗУЧАЕМ RIDES
# ==========================================

In [8]:

print("📊 RIDES DATASET")
print("="*90)

# Первые 5 строк
print("\n🔍 Первые строки:")
rides.head()


📊 RIDES DATASET

🔍 Первые строки:


,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [9]:
print("\n📋 Информация о колонках:")
rides.info()



📋 Информация о колонках:
<class 'pandas.DataFrame'>
RangeIndex: 693071 entries, 0 to 693070
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   distance          693071 non-null  float64
 1   cab_type          693071 non-null  str    
 2   time_stamp        693071 non-null  int64  
 3   destination       693071 non-null  str    
 4   source            693071 non-null  str    
 5   price             637976 non-null  float64
 6   surge_multiplier  693071 non-null  float64
 7   id                693071 non-null  str    
 8   product_id        693071 non-null  str    
 9   name              693071 non-null  str    
dtypes: float64(3), int64(1), str(6)
memory usage: 52.9 MB


In [10]:
print("\n📈 Статистика:")
rides.describe()


📈 Статистика:


,distance,time_stamp,price,surge_multiplier
count,693071.000000,6.930710e+05,637976.000000,693071.000000
mean,2.189430,1.544046e+12,16.545125,1.013870
std,1.138937,6.891925e+08,9.324359,0.091641
min,0.020000,1.543204e+12,2.500000,1.000000
25%,1.280000,1.543444e+12,9.000000,1.000000
50%,2.160000,1.543737e+12,13.500000,1.000000
75%,2.920000,1.544828e+12,22.500000,1.000000
max,7.860000,1.545161e+12,97.500000,3.000000


In [11]:
print("\n⚠️ Missing values:")
missing = rides.isnull().sum()
print(missing[missing > 0])


⚠️ Missing values:
price    55095
dtype: int64


# ==========================================
# БЛОК 1.2: ИЗУЧАЕМ WEATHER
# ==========================================

In [12]:

print("📊 Weather DATASET")
print("="*90)

# Первые 5 строк
print("\n🔍 Первые строки:")
weather.head()

📊 Weather DATASET

🔍 Первые строки:


,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,1545003901,0.76,11.32
2,42.50,Boston University,1.0,1012.15,0.1089,1545003901,0.76,11.07
3,42.11,Fenway,1.0,1012.13,0.0969,1545003901,0.77,11.09
4,43.13,Financial District,1.0,1012.14,0.1786,1545003901,0.75,11.49


In [13]:
print("\n📋 Информация о колонках:")
weather.info()


📋 Информация о колонках:
<class 'pandas.DataFrame'>
RangeIndex: 6276 entries, 0 to 6275
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp        6276 non-null   float64
 1   location    6276 non-null   str    
 2   clouds      6276 non-null   float64
 3   pressure    6276 non-null   float64
 4   rain        894 non-null    float64
 5   time_stamp  6276 non-null   int64  
 6   humidity    6276 non-null   float64
 7   wind        6276 non-null   float64
dtypes: float64(6), int64(1), str(1)
memory usage: 392.4 KB


In [14]:
print("\n📈 Статистика:")
weather.describe().round(2)


📈 Статистика:


,temp,clouds,pressure,rain,time_stamp,humidity,wind
count,6276.00,6276.00,6276.00,894.00,6.276000e+03,6276.00,6276.00
mean,39.09,0.68,1008.45,0.06,1.543857e+09,0.76,6.80
std,6.02,0.31,12.87,0.10,6.659340e+05,0.13,3.63
min,19.62,0.00,988.25,0.00,1.543204e+09,0.45,0.29
25%,36.08,0.44,997.75,0.00,1.543387e+09,0.67,3.52
50%,40.13,0.78,1007.66,0.01,1.543514e+09,0.76,6.57
75%,42.83,0.97,1018.48,0.06,1.544691e+09,0.89,9.92
max,55.41,1.00,1035.12,0.78,1.545159e+09,0.99,18.18


In [15]:
print("\n⚠️ Missing values:")
missing_weather = weather.isnull().sum()
print(missing_weather[missing_weather > 0])


⚠️ Missing values:
rain    5382
dtype: int64


In [16]:
# ==========================================
# БЛОК 1.3: Приводим данные в порядок
# ==========================================

In [17]:
weather['time_stamp'] = pd.to_datetime(weather['time_stamp'], unit='s')
weather.head()

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,2018-12-16 23:45:01,0.77,11.25
1,42.43,Beacon Hill,1.0,1012.15,0.1846,2018-12-16 23:45:01,0.76,11.32
2,42.50,Boston University,1.0,1012.15,0.1089,2018-12-16 23:45:01,0.76,11.07
3,42.11,Fenway,1.0,1012.13,0.0969,2018-12-16 23:45:01,0.77,11.09
4,43.13,Financial District,1.0,1012.14,0.1786,2018-12-16 23:45:01,0.75,11.49


In [18]:
rides['time_stamp'] = pd.to_datetime(rides['time_stamp'], unit='ms')
rides.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,2018-12-16 09:30:07.890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared
1,0.44,Lyft,2018-11-27 02:00:23.677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux
2,0.44,Lyft,2018-11-28 01:00:22.198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft
3,0.44,Lyft,2018-11-30 04:53:02.749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL
4,0.44,Lyft,2018-11-29 03:49:20.223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL


In [ ]:
rides['year'] = rides['time_stamp'].dt.year
rides['month'] = rides['time_stamp'].dt.month
rides['month_name'] = rides['time_stamp'].dt.month_name()
rides['day'] = rides['time_stamp'].dt.day
rides['day_name'] = rides['time_stamp'].dt.day_name()
rides['hour'] = rides['time_stamp'].dt.hour



,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,year,month,month_name,day,day_name,hour
0,0.44,Lyft,2018-12-16 09:30:07.890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018,12,December,16,Sunday,9
1,0.44,Lyft,2018-11-27 02:00:23.677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018,11,November,27,Tuesday,2
2,0.44,Lyft,2018-11-28 01:00:22.198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,2018,11,November,28,Wednesday,1
3,0.44,Lyft,2018-11-30 04:53:02.749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,2018,11,November,30,Friday,4
4,0.44,Lyft,2018-11-29 03:49:20.223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,2018,11,November,29,Thursday,3


In [31]:
rides['price'] = rides['price'].fillna(rides.groupby('name')['price'].transform('median'))

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,year,month,month_name,day,day_name,hour
0,0.44,Lyft,2018-12-16 09:30:07.890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared,2018,12,December,16,Sunday,9
1,0.44,Lyft,2018-11-27 02:00:23.677,North Station,Haymarket Square,11.0,1.0,4bd23055-6827-41c6-b23b-3c491f24e74d,lyft_premier,Lux,2018,11,November,27,Tuesday,2
2,0.44,Lyft,2018-11-28 01:00:22.198,North Station,Haymarket Square,7.0,1.0,981a3613-77af-4620-a42a-0c0866077d1e,lyft,Lyft,2018,11,November,28,Wednesday,1
3,0.44,Lyft,2018-11-30 04:53:02.749,North Station,Haymarket Square,26.0,1.0,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,lyft_luxsuv,Lux Black XL,2018,11,November,30,Friday,4
4,0.44,Lyft,2018-11-29 03:49:20.223,North Station,Haymarket Square,9.0,1.0,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,lyft_plus,Lyft XL,2018,11,November,29,Thursday,3


In [30]:
weather['rain'] = weather['rain'].fillna(weather['rain'].median())